In [6]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

## reading ManhattanLotLocs data into lot_df DataFrame

lot_df = pd.DataFrame(pd.read_csv('data/ManhattanLotLocs.txt'))

In [7]:
lot_df.head(10)

,lot_id,Block,Lot,Address,ZipCode,latitude,longitude
0,1_10,1,10,1 GOVERNORS ISLAND,10004,40.688799,-74.018675
1,1_101,1,101,1 LIBERTY ISLAND,10004,40.689920,-74.045337
2,1_201,1,201,1 ELLIS ISLAND,10004,40.698188,-74.041329
3,2_1,2,1,MARGINAL STREET,10004,40.700369,-74.012911
4,2_2,2,2,10 SOUTH STREET,10004,40.700550,-74.011588
5,2_23,2,23,1 PIER 6,10004,40.701171,-74.009243
6,3_1,3,1,10 BATTERY PARK,10280,40.702806,-74.015631
7,3_3,3,3,PETER MINUIT PLAZA,10004,40.701953,-74.013146
8,3_10,3,10,1 SOUTH STREET,10004,40.700918,-74.014444
9,4_7501,4,7501,1 WATER STREET,10004,40.702153,-74.012021


In [8]:
len(lot_df['lot_id'].unique())

42396

In [9]:
lot_df.shape ## all lot_id values are unique

(42396, 7)

In [10]:
lotdf_colindex = lot_df.columns.values
lotdf_colindex

array(['lot_id', 'Block', 'Lot', 'Address', 'ZipCode', 'latitude',
       'longitude'], dtype=object)

In [11]:
len(lot_df['Lot'].unique()) ## lot id's are composed of block_lot 

544

In [12]:
lot_id_df = lot_df.set_index(lot_df['lot_id'])
# lot_id_df.head(10)
lot_id_df = lot_id_df.drop('lot_id', 1)
lot_id_df.head()

## lot_id_df is lot_df indexed by lot_ids 

,Block,Lot,Address,ZipCode,latitude,longitude
lot_id,,,,,,
1_10,1,10,1 GOVERNORS ISLAND,10004,40.688799,-74.018675
1_101,1,101,1 LIBERTY ISLAND,10004,40.689920,-74.045337
1_201,1,201,1 ELLIS ISLAND,10004,40.698188,-74.041329
2_1,2,1,MARGINAL STREET,10004,40.700369,-74.012911
2_2,2,2,10 SOUTH STREET,10004,40.700550,-74.011588


### Calculate Manhattan Distance for each lot to the each pizza shop in miles

In [14]:
## Load Pizza Map location data (from Ksenia)

pizza_df = pd.DataFrame(pd.read_csv('data/pizza_locations_google.csv'))
# pizza_df.head(10)

pizza_df = pizza_df.set_index('Unnamed: 0', drop=True)

pizza_df_colindex = pizza_df.columns.values
# pizza_df_colindex

pizza_df.head()

,name,lat,lng,vicinity,open_now,rating,price_level,icon,id,photos,place_id,reference,scope,types,borough
Unnamed: 0,,,,,,,,,,,,,,,
0,P&M Classic Pizza and Restaurant,40.826154,-73.946575,"1747 Amsterdam Avenue, New York",False,4.7,1.0,https://maps.gstatic.com/mapfiles/place_api/ic...,31f44d5c640ad7d4737047373e4cf4dbd957f173,"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJqX832Xz2wokRCmDaupc-83M,CmRRAAAAOywdi6JqBTmF-VTLLJweUJOPHva0nHoXCyydOa...,GOOGLE,"['restaurant', 'food', 'point_of_interest', 'e...",NYC
1,New Broadway Pizza Coffee Shop,40.851788,-73.934708,"4312 Broadway, New York",False,4.4,2.0,https://maps.gstatic.com/mapfiles/place_api/ic...,e438e1ef8e28dfd3ebf583260d76259e238b815e,"[{'height': 4176, 'html_attributions': ['<a hr...",ChIJhx0DCh70wokRKr1DakdaiKo,CmRSAAAAriFU3PPuVv34QPIZBsAbrhzJoaURZCUDlUMNc8...,GOOGLE,"['restaurant', 'food', 'point_of_interest', 'e...",NYC
2,Tommy's Grill & Pizzeria,40.833109,-73.945095,"3750 Broadway, New York",True,3.2,1.0,https://maps.gstatic.com/mapfiles/place_api/ic...,2f47167bd8c672824a1123577e60362b74d48a6f,"[{'height': 540, 'html_attributions': ['<a hre...",ChIJrdYyy4X2wokR8AWLcUl4usA,CmRSAAAAeBpQ1xeDg9hXAA94b1hLidMs_z-lBo0uAHhPHr...,GOOGLE,"['meal_delivery', 'meal_takeaway', 'restaurant...",NYC
3,Custom Fuel Pizza,40.808704,-73.952025,"2288 Frederick Douglass Boulevard, New York",False,4.0,1.0,https://maps.gstatic.com/mapfiles/place_api/ic...,ac91f24418e1683a204b1268a0fe3b9c76874a5c,"[{'height': 750, 'html_attributions': ['<a hre...",ChIJ57jXPxP2wokRRrcwDoksk18,CmRRAAAAuG9Cxht8Cz8ge_29uFCFSXhMFpk2VMVpRkrKS3...,GOOGLE,"['bar', 'restaurant', 'food', 'point_of_intere...",NYC
4,Pizza Stop,40.815397,-73.940027,"531 Lenox Avenue, New York",True,4.0,2.0,https://maps.gstatic.com/mapfiles/place_api/ic...,72e47bb5001b6f0879bc827fe268fd0d868e5ddd,"[{'height': 419, 'html_attributions': ['<a hre...",ChIJy6G-JHj2wokRiLxKme5PyBQ,CmRRAAAAjBlEWBE8Mc2YshcH9x8ZVLvAiE-hzRqZ5ZnRGX...,GOOGLE,"['meal_delivery', 'meal_takeaway', 'restaurant...",NYC


In [10]:
## Manhattan Distance Function

## nearest listing in terms of 1) Specified Distance (user input function) 
                            ## 2) Specified Number of Pizza Shops
                            ## 3) Specified Neighborhood

In [11]:
## DataFrame: rows = lots, columns = pizza shops

In [15]:
## Creating a DataFrame to store lattitude-longitude data for Manhattan Lots
lot_indexvector = [str(k) for k in lot_id_df.index.values]
lot_lat_vector = lot_id_df['latitude']
lot_lng_vector = lot_id_df['longitude']
zipcode_vector = lot_id_df['ZipCode']

lot_latlng = pd.DataFrame({'latitude': lot_lat_vector, 'longitude':lot_lng_vector, 'ZipCode':zipcode_vector}, 
                          index=lot_indexvector)

lot_latlng.head()

,ZipCode,latitude,longitude
1_10,10004,40.688799,-74.018675
1_101,10004,40.689920,-74.045337
1_201,10004,40.698188,-74.041329
2_1,10004,40.700369,-74.012911
2_2,10004,40.700550,-74.011588


In [16]:
## Creating a DataFrame to store lattitude-longitude data for Manhattan Pizza Shops
pizza_indexvector = pizza_df.index.values
pizza_lat_vector = pizza_df['lat']
pizza_lng_vector = pizza_df['lng']

pizza_latlng = pd.DataFrame({'latitude': pizza_lat_vector, 'longitude': pizza_lng_vector},
                           index=pizza_indexvector)

pizza_latlng['pizza_id'] = pizza_df['place_id']
pizza_latlng['pizza_id'] = [str(k) for k in pizza_latlng['pizza_id']]
pizza_latlng = pizza_latlng.set_index('pizza_id', drop=True)
pizza_latlng.head()

,latitude,longitude
pizza_id,,
ChIJqX832Xz2wokRCmDaupc-83M,40.826154,-73.946575
ChIJhx0DCh70wokRKr1DakdaiKo,40.851788,-73.934708
ChIJrdYyy4X2wokR8AWLcUl4usA,40.833109,-73.945095
ChIJ57jXPxP2wokRRrcwDoksk18,40.808704,-73.952025
ChIJy6G-JHj2wokRiLxKme5PyBQ,40.815397,-73.940027


In [17]:
### Merging pizza_latlng df and lot_latlng df on latlngs to obtain zipcodes 

merged_data = pd.merge(lot_latlng, pizza_latlng, how='outer', on=['latitude', 'longitude'])

## prints a dataframe with average latlong data for each zipcode
merged_data.groupby('ZipCode').mean().head(15)

average_latlng = merged_data.groupby('ZipCode').mean()
average_latlng

,latitude,longitude
ZipCode,,
10001.0,40.748747,-73.994244
10002.0,40.717181,-73.989813
10003.0,40.731315,-73.988127
10004.0,40.703878,-74.012221
10005.0,40.706392,-74.008407
10006.0,40.708385,-74.013442
10007.0,40.714298,-74.008373
10009.0,40.725521,-73.981106
10010.0,40.739990,-73.985585


In [18]:
Central_Harlem = [10026, 10027, 10030, 10037, 10039]
Chelsea_and_Clinton = [10001, 10011, 10018, 10019, 10020, 10036]
East_Harlem =[10029, 10035]
Gramercy_Park_and_Murray_Hill= [10010, 10016, 10017, 10022]
Greenwich_Village_and_Soho = [10012, 10013, 10014]
Lower_Manhattan = [10004, 10005, 10006, 10007, 10038, 10280]
Lower_East_Side = [10002, 10003, 10009]
Upper_East_Side = [10021, 10028, 10044, 10065, 10075, 10128]
Upper_West_Side = [10023, 10024, 10025]
Inwood_and_Washington_Heights = [10031, 10032, 10033, 10034, 10040]

### Manhattan Distance with Haversine Formula

Haversine Formula is used to calculate the distance of two points on a sphere given their latitudes and longitudes

a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2) 

c = 2 ⋅ asin( √a )

d = R ⋅ c

where, R = 6371 km (3959 miles) (Earth's mean radius), φ is latitude, λ is longitude. 



In [19]:
from math import sin
from math import cos
from math import atan2
from math import sqrt
from math import asin
from math import radians


def ManhattanDist(loc_lot, loc_pizza):
    '''Takes as input a lot_id and a pizza_id to calculate the distance between the
    two points with the Haversine formula. Returns a dataframe wtih rows = lot_ids, 
    columns = pizza_ids, data = distance in kms'''
    
    lot_lat = lot_latlng.loc[loc_lot]['latitude']
    lot_lng = lot_latlng.loc[loc_lot]['longitude']
    pizza_lat = pizza_latlng.loc[loc_pizza]['latitude']
    pizza_lng = pizza_latlng.loc[loc_pizza]['longitude']
    
    r = 3959
    
### If Manhattan was aligned with N-S 

#     longitude_a = sin(0)**2 + cos(radians(lot_lat))*cos(radians(lot_lat))*sin(radians((lot_lng-pizza_lng)/2))**2
    
#     latitude_a = (sin(radians(abs((lot_lat-pizza_lat)/2))))**2 + cos(radians(lot_lat))*cos(pizza_lat)*(sin(0))**2
    
#     longitude_c = c = 2*asin(sqrt(longitude_a))
                                                                              
#     latitude_c = 2*asin(sqrt(latitude_a))
                                                                              
#     d_latitude = r*latitude_c
                                                                              
#     d_longitude = r*longitude_c
                                                                              
#     d = abs(d_latitude) + abs(d_longitude)
                                                                              
#     return d 

    a = sin((radians(abs(lot_lat-pizza_lat)))/2)**2 + cos(lot_lat)*cos(pizza_lat)*sin((radians(abs(lot_lng - pizza_lng)/2)))**2

    c = 2*asin(sqrt(a))         
    
    d = r*c
    
    return d
                                                                                       
print (ManhattanDist('2_2', 'ChIJy6G-JHj2wokRiLxKme5PyBQ'))

print (ManhattanDist('3_10', 'ChIJy6G-JHj2wokRiLxKme5PyBQ'))

print (ManhattanDist('7_29', 'ChIJ57jXPxP2wokRRrcwDoksk18'))

print (ManhattanDist('3_1', 'ChIJbWfIg3j2wokRJ22QKbeFRu8'))

9.336840540537995
9.420523715895092
8.343918810496316
9.765088632893365


In [23]:
def list_distance(pizza):
    Pizza_Dict = {}
    for lot in lot_latlng.index:
        Pizza_Dict[str(lot)] = ManhattanDist(lot, pizza)
    return Pizza_Dict 

distance_df = pd.DataFrame()
pizza_dict_to_df = [list_distance(pizza) for pizza in pizza_df['place_id']]
# distance_df.index = pizza_dict_to_df[0].keys()
distance_df = distance_df.append(pizza_dict_to_df)
distance_df['place_id'] = pizza_df['place_id']
distance_df = distance_df.set_index('place_id', drop=True)
distance_df.head()

# 
# distance_df['place_id'] = pizza_df['place_id']
# distance_df.set_index('place')
distance_df.to_csv('lot_to_pizza.csv')

KeyboardInterrupt: 

In [24]:
from math import sin
from math import cos
from math import atan2
from math import sqrt
from math import asin
from math import radians


def ZipManhattanDist(zip_loc, loc_pizza):
    '''Takes as input a lot_id and a pizza_id to calculate the distance between the
    two points with the Haversine formula. Returns a dataframe wtih rows = lot_ids, 
    columns = pizza_ids, data = distance in kms'''
    
    zip_lat = average_latlng.loc[zip_loc]['latitude']
    zip_lng = average_latlng.loc[zip_loc]['longitude']
    pizza_lat = pizza_latlng.loc[loc_pizza]['latitude']
    pizza_lng = pizza_latlng.loc[loc_pizza]['longitude']
    
    r = 3959
    a = sin((radians(abs(zip_lat-pizza_lat)))/2)**2 + cos(zip_lat)*cos(pizza_lat)*sin((radians(abs(zip_lng - pizza_lng)/2)))**2
    c = 2*asin(sqrt(a))         
    d = r*c
    return d
                                                                                       
print (ZipManhattanDist(10003.0, 'ChIJy6G-JHj2wokRiLxKme5PyBQ'))



6.6881085770670134


In [25]:
## Calculates the pizza place distances to each zipcode's average latlng

def zipcode_dist(pizza):
    Pizza_Dict = {}
    for zipcode in average_latlng.index:
        Pizza_Dict[zipcode] = ZipManhattanDist(zipcode, pizza)
    return Pizza_Dict 

distance_df = pd.DataFrame()
pizza_dict_to_df = [zipcode_dist(pizza) for pizza in pizza_df['place_id']]
# distance_df.index = pizza_dict_to_df[0].keys()
distance_df = distance_df.append(pizza_dict_to_df)
distance_df['place_id'] = pizza_df['place_id']
distance_df = distance_df.set_index('place_id', drop=True)

distance_df
distance_df.to_csv('pizza_zipcode_distance.csv')


In [34]:
distance_df.loc['Averages'] = distance_df.describe().loc['mean'].values
distance_df.to_csv('Zipcode_Avg_Dist.csv')


In [83]:
### Masking the dataframe to get closest distances
def get_closest_pizza(zipcode): 
    
    close_pizzas = distance_df[[zipcode]].apply(lambda x: x <=2)
    pizza_ids = close_pizzas[close_pizzas[zipcode] == True].index.values
    return pizza_ids

zipcode_dict = {}
for zipcode in distance_df.columns:
    zipcode_dict[str(zipcode).replace('.0', '')] = get_closest_pizza(zipcode)
    
zipcode_dict['10003']  ## contains the place_ids for pizza stores within 2 miles of each zipcode's average latitude longitude

array(['ChIJIcJCN1RYwokRHQzLh_iGQwI', 'ChIJCxdJV5hZwokRoBWzilBPDMs',
       'ChIJ_1bEhaxZwokRIqNQV6dunL8', 'ChIJsS_-2rFZwokRWYvlpNNbjmg',
       'ChIJyxltMANZwokRJ6SOzQ1-Mf4', 'ChIJLWzY2wNZwokR_fsvuheHrfU',
       'ChIJ6RV0FqRZwokR4QupKOKzxsE', 'ChIJ5Q-Ra6BZwokRd1gGerJ74s8',
       'ChIJcfHLy5BZwokRafTR9FHrpNw', 'ChIJGck76lRYwokROSMFvkw6UWM',
       'ChIJj5R4TqhZwokRuxkrhtALBKk', 'ChIJ42SVvf9YwokRD35keDIYqj4',
       'ChIJ44UCRgBZwokRPt7jjyziLpY', 'ChIJBWWCLQBZwokRBrGegvRm9qg',
       'ChIJq3gz6plZwokR1WPh3L5frsc', 'ChIJybOUCJlZwokRxiXBjbhX-iA',
       'ChIJBSESh1RYwokRyHcVnrG7JWo', 'ChIJXT6bwpVZwokRvbyM5uKN_G4',
       'ChIJURgSuVJYwokRyUTyhXB9K4c', 'ChIJ84bLVbpZwokRMdylFSXlstU',
       'ChIJt8lZE6ZZwokRB4hefl-4Iek', 'ChIJZzfElKVZwokRIem7YxXHhcw',
       'ChIJJ07QdrdZwokRXdz8zv2_eg4', 'ChIJnQT_1rFZwokRJe2ZG3BCumY',
       'ChIJPxPd_P1YwokRfzLhSiACEoU', 'ChIJB2Ufwq1ZwokR1tfIL39GOUQ',
       'ChIJzxso-AVZwokRw2RNQ91j6L0', 'ChIJNf2Jg7xZwokREAj--Xq1cFY',
       'ChIJFe1v6wVZwokR9IkbpwFaK-

In [82]:
pizza_latlng.head()

,latitude,longitude
pizza_id,,
ChIJqX832Xz2wokRCmDaupc-83M,40.826154,-73.946575
ChIJhx0DCh70wokRKr1DakdaiKo,40.851788,-73.934708
ChIJrdYyy4X2wokR8AWLcUl4usA,40.833109,-73.945095
ChIJ57jXPxP2wokRRrcwDoksk18,40.808704,-73.952025
ChIJy6G-JHj2wokRiLxKme5PyBQ,40.815397,-73.940027


In [117]:
### Masking the dataframe to get closest distances
def get_closest_zipcode(pizza): 
    min_zip_dict = {}
    close_zipcodes = distance_df.T[[pizza]].apply(lambda x: min(x))
    zipcode_min = close_zipcodes[close_zipcodes[pizza] == True]
    return zipcode_min

pizza_zipcode_dict = {}
for pizza in distance_df.T.columns:
    pizza_zipcode_dict[str(pizza)] = get_closest_zipcode(pizza)
    
pizza_zipcode_dict ## maps each pizza place to its closest zipcode latlng average

closest = {}
for pizza_id in pizza_zipcode_dict.keys():
    closest[pizza_id] = distance_df.T.sort_values(by=pizza_id, ascending=True)[pizza].index[0]
#     print (pizza_id, (distance_df.T.sort_values(by=pizza_id, ascending=True)[pizza]).index[0])

closest

{'ChIJ-4YKz4pYwokR8E_PydQfOCE': 10023.0,
 'ChIJ-VLiKPpYwokR6WqPBMALoAI': 10103.0,
 'ChIJ0wWJG-VYwokRdeKRNMvIHN4': 10022.0,
 'ChIJ1Z3RurNZwokROf19wP13fcs': 10001.0,
 'ChIJ37TJUxdawokRQUvJ1X8oeyo': 10006.0,
 'ChIJ39rtLCb2wokRNA_Dd5ro2Xo': 10025.0,
 'ChIJ3Wu2ooT2wokRztqh0rqMNeY': 10032.0,
 'ChIJ42SVvf9YwokRD35keDIYqj4': 10020.0,
 'ChIJ44UCRgBZwokRPt7jjyziLpY': 10119.0,
 'ChIJ57jXPxP2wokRRrcwDoksk18': 10027.0,
 'ChIJ5Q-Ra6BZwokRd1gGerJ74s8': 10010.0,
 'ChIJ5Qe7Jb5YwokRCxZzLKCneyQ': 10075.0,
 'ChIJ5bwuMFdYwokRliCt_Efmc3U': 10019.0,
 'ChIJ6RV0FqRZwokR4QupKOKzxsE': 10010.0,
 'ChIJ6ffdpJNZwokRmcafdROM5q0': 10014.0,
 'ChIJ6xvs94VZwokRnT1D2lX2OTw': 10012.0,
 'ChIJ84bLVbpZwokRMdylFSXlstU': 10001.0,
 'ChIJ8Q2WSpJZwokRQz-bYYgEskM': 10014.0,
 'ChIJ8dFzuAf2wokRuX_Ly_NBc4A': 10035.0,
 'ChIJ8yF04LhYwokRqO1yxp4iSQA': 10028.0,
 'ChIJ90r0N8BYwokRT_Pu0DkD8Hk': 10021.0,
 'ChIJ94FpoWL2wokRpvuJQ5m5Z5g': 10031.0,
 'ChIJAxR92opYwokR58ToOJcEkyk': 10023.0,
 'ChIJB2Ufwq1ZwokR1tfIL39GOUQ': 10123.0,
 'ChIJBRtDt-dYwo